- The third-hardest problem in physics is finding a proper wine to celebrate discoveries. Load the wine data from chapter 4, and create a new model with the appropriate number of input parameters.
    - How long does it take to train compared to the temperature data we have been using?
    - Can you explain what factors contribute to the training times?
    - Can you get the loss to decrease while training on this dataset?
    - How would you go about graphing this dataset?
- まずload data:

In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [5]:
wine_path = '../data/p1ch4/tabular-wine/winequality-white.csv'
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=';',
                        skiprows=1)
wineq = torch.from_numpy(wineq_numpy)
data = wineq[:, :-1]
target = wineq[:, -1]

In [6]:
data.shape, target.shape

(torch.Size([4898, 11]), torch.Size([4898]))

- 取りあえずtargetになっているqualityはcontinuous valueと見る。
- dataをnormalizeする：

In [7]:
data_mean = torch.mean(data, dim=0)
data_var = torch.var(data, dim=0)
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])

- targetをunsqueezeする：

In [8]:
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed.shape

torch.Size([4898, 1])

- training, validation dataに分ける：

In [12]:
n_samples = data.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

data_train = data_normalized[train_indices]
target_train = target_unsqueezed[train_indices]

data_val = data_normalized[val_indices]
target_val = target_unsqueezed[val_indices]

data_train.shape, data_val.shape

(torch.Size([3919, 11]), torch.Size([979, 11]))

- model, loss, training loop:

In [10]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)
        
        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        
        if epoch == 1 or epoch % 1000 == 0:
            print(f'Epoch {epoch}, Training loss {loss_train.item():.4f},'
                 f' Validation loss {loss_val.item():.4f}')

In [13]:
seq_model = nn.Sequential(nn.Linear(11, 20),
                         nn.Tanh(),
                         nn.Linear(20, 1))
optimizer = optim.SGD(seq_model.parameters(), lr=1e-3)

training_loop(n_epochs=5000,
             optimizer=optimizer,
             model=seq_model,
             loss_fn=nn.MSELoss(),
             t_u_train=data_train,
             t_u_val=data_val,
             t_c_train=target_train,
             t_c_val=target_val)

Epoch 1, Training loss 35.1173, Validation loss 34.7226
Epoch 1000, Training loss 0.5710, Validation loss 0.5606
Epoch 2000, Training loss 0.5558, Validation loss 0.5472
Epoch 3000, Training loss 0.5500, Validation loss 0.5415
Epoch 4000, Training loss 0.5454, Validation loss 0.5370
Epoch 5000, Training loss 0.5416, Validation loss 0.5332


- 学習時間は明らかに長くなった。具体的な時間は図っていない。
- input featureが11になったこと、学習データが多くなったことで、学習時間が長くなったでしょう。
- featureが11なので、温度計の例のような可視化はできない。普段はtraining lossやvalidation lossをplotするでしょう。